In [ ]:
import os, json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler

npz = np.load("../artifacts/dataset_windows.npz", allow_pickle=True)
X_train = npz["X_train"]; Y_train = npz["Y_train"]
X_val = npz["X_val"]; Y_val = npz["Y_val"]
X_test = npz["X_test"]; Y_test = npz["Y_test"]
features = list(npz["features"])
input_hours = int(npz["input_hours"])
horizon = int(npz["horizon"])

X_train.shape, Y_train.shape, features, input_hours, horizon

In [ ]:
scaler = StandardScaler()

def fit_scaler(X):
    flat = X.reshape(-1, X.shape[-1])
    scaler.fit(flat)

def transform(X):
    flat = X.reshape(-1, X.shape[-1])
    flat2 = scaler.transform(flat).astype("float32")
    return flat2.reshape(X.shape)

fit_scaler(X_train)
Xtr = transform(X_train)
Xva = transform(X_val)
Xte = transform(X_test)

Xtr.shape

In [ ]:
tf.keras.backend.clear_session()

model = keras.Sequential([
    layers.Input(shape=(input_hours, Xtr.shape[-1])),
    layers.GRU(64, return_sequences=True),
    layers.Dropout(0.15),
    layers.GRU(64),
    layers.Dropout(0.15),
    layers.Dense(64, activation="relu"),
    layers.Dense(horizon)
])

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss="mse",
    metrics=[keras.metrics.MeanAbsoluteError(name="mae")]
)

model.summary()

In [ ]:
callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_mae", patience=8, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor="val_mae", patience=4, factor=0.5, min_lr=1e-5)
]

history = model.fit(
    Xtr, Y_train,
    validation_data=(Xva, Y_val),
    epochs=60,
    batch_size=64,
    callbacks=callbacks,
    verbose=1
)

pd.DataFrame(history.history).tail()

In [ ]:
hist = pd.DataFrame(history.history)

plt.figure(figsize=(10,4))
plt.plot(hist["loss"], label="train loss")
plt.plot(hist["val_loss"], label="val loss")
plt.legend(); plt.title("Loss (MSE)"); plt.tight_layout(); plt.show()

plt.figure(figsize=(10,4))
plt.plot(hist["mae"], label="train MAE")
plt.plot(hist["val_mae"], label="val MAE")
plt.legend(); plt.title("MAE"); plt.tight_layout(); plt.show()

In [ ]:
pred = model.predict(Xte, verbose=0)

mae = mean_absolute_error(Y_test.flatten(), pred.flatten())
rmse = mean_squared_error(Y_test.flatten(), pred.flatten(), squared=False)

print("TEST MAE:", mae)
print("TEST RMSE:", rmse)

plt.figure(figsize=(6,6))
plt.scatter(Y_test.flatten(), pred.flatten(), s=4)
plt.xlabel("Real temp"); plt.ylabel("Pred temp")
plt.title("Prediction vs Real (all horizons)")
mn = min(Y_test.min(), pred.min()); mx = max(Y_test.max(), pred.max())
plt.plot([mn,mx],[mn,mx])
plt.tight_layout(); plt.show()

In [ ]:
k = min(10, len(Xte)-1)
real = Y_test[k]
p = pred[k]

plt.figure(figsize=(10,4))
plt.plot(real, label="real")
plt.plot(p, label="pred")
plt.title("One sample: 24h temperature forecast")
plt.legend(); plt.tight_layout(); plt.show()

In [ ]:
os.makedirs("../artifacts", exist_ok=True)
model_path = "../artifacts/model.h5"
model.save(model_path)
print("Saved:", model_path)

In [ ]:
os.makedirs("../web", exist_ok=True)

scaler_payload = {
    "features": features,
    "input_hours": input_hours,
    "horizon": horizon,
    "mean": scaler.mean_.tolist(),
    "scale": scaler.scale_.tolist(),
    "note": "StandardScaler fitted on training set (flattened time dimension)."
}

with open("../web/scaler.json", "w", encoding="utf-8") as f:
    json.dump(scaler_payload, f, ensure_ascii=False, indent=2)

print("Saved: ../web/scaler.json")